In [0]:
!pip install syft

In [0]:
import torch as th
from torch import nn, optim
import syft as sy
hook = sy.TorchHook(th)
alice = sy.VirtualWorker(hook, id="alice")
bob = sy.VirtualWorker(hook, id="bob")
secure_worker = sy.VirtualWorker(hook, id="secure_worker")
alice.add_workers([bob, secure_worker])
bob.add_workers([alice, secure_worker])
secure_worker.add_workers([bob, alice])

W0725 06:44:54.146835 139711681836928 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0725 06:44:54.163732 139711681836928 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W0725 06:44:54.801971 139711681836928 base.py:628] Worker bob already exists. Replacing old worker which could cause                     unexpected behavior
W0725 06:44:54.802900 139711681836928 base.py:628] Worker secure_worker already exists. Replacing old worker which could cause                     unexpected behavior
W0725 06:44:54.804019 139711681836928 base.py:628] Worker alice already exists. Replacing old worker which could 

<VirtualWorker id:secure_worker #objects:0>

In [0]:
alice.clear_objects()
bob.clear_objects()
secure_worker.clear_objects()

<VirtualWorker id:secure_worker #objects:0>

In [0]:
data = th.tensor([[1.,1],[0,1],[1,0],[0,0]], requires_grad = True)
target = th.tensor([[1.],[1],[0],[0]], requires_grad = True)

In [0]:
alice_data = data[0:2].send(alice)
alice_target = target[0:2].send(alice)

bob_data = data[2:].send(bob)
bob_target = target[2:].send(bob)

In [0]:
bob._objects

{80866162505: tensor([[0.],
         [0.]], requires_grad=True), 93288834077: tensor([[1., 0.],
         [0., 0.]], requires_grad=True)}

In [0]:
alice._objects

{13504270611: tensor([[1., 1.],
         [0., 1.]], requires_grad=True), 95383268194: tensor([[1.],
         [1.]], requires_grad=True)}

In [0]:
model = nn.Linear(2,1)

In [0]:
for i in range(10):
  alice_model = model.copy().send(alice)
  bob_model = model.copy().send(bob)
  
  alice_opt = optim.SGD(params=alice_model.parameters(), lr=0.01)
  bob_opt = optim.SGD(params=bob_model.parameters(), lr=0.01)
  
  #params = [list(bob_model.parameters()), list(alice_model.parameters())]
  
  for worker_iter in range(5):
    alice_opt.zero_grad()
    alice_pred = alice_model(alice_data)
    alice_loss = ((alice_pred - alice_target) ** 2).sum()
    alice_loss.backward()
    alice_opt.step()
    alice_loss = alice_loss.get().data
    
    bob_opt.zero_grad()
    bob_pred = bob_model(bob_data)
    bob_loss = ((bob_pred - bob_target) ** 2).sum()
    bob_loss.backward()
    bob_opt.step()
    bob_loss = bob_loss.get().data
  
  '''for remote_index in range(len(compute_nodes)):
    
    if(remote_index == 0):
      data, target = bob_data, bob_target
    else:
      data, target = alice_data, alice_target
      
    models[remote_index] = update(data, target, models[remote_index], optimizers[remote_index])
    
  new_params = list()
  for param_i in range(len(params[0])):
    worker_param = list()
    for index in range(len(compute_nodes)):
      encrypted_param = params[index][param_i].copy().fix_prec().share(bob, alice, crypto_provider = secure_worker)
      worker_param.append(encrypted_param.get())
      
    new_param = (worker_param[0] + worker_param[1]).get().float_prec()/2
    new_params.append(new_param)'''
  
  alice_wt = alice_model.weight.data.fix_precision().share(bob, alice, crypto_provider=secure_worker).get()
  bob_wt = bob_model.weight.data.fix_precision().share(bob, alice, crypto_provider = secure_worker).get()
  alice_bias = alice_model.bias.data.fix_precision().share(bob, alice, crypto_provider = secure_worker).get() 
  bob_bias = bob_model.bias.data.fix_precision().share(bob, alice, crypto_provider = secure_worker).get()
  
  with th.no_grad():
    model.weight.set_((( alice_wt + bob_wt )/2).get().float_prec())
    model.bias.set_((( alice_bias + bob_bias)/2).get().float_prec())
    
    '''for index in range(len(compute_nodes)):
      for param_i in range(len(params[index])):
        params[index][param_i].set_(new_params[param_i])'''
    
  print("Alice's loss:" + str(alice_loss) + "  Bob's loss:" + str(bob_loss))

AttributeError: ignored